In [ ]:
# Based on this paper - https://arxiv.org/pdf/1706.03762.pdf
# Might want to move layer norm inside the residual block - https://arxiv.org/pdf/2002.04745.pdf
# Layer normalization - https://arxiv.org/pdf/1607.06450.pdf
# TODO: Investigate learning rate warmup - https://arxiv.org/abs/2002.04745
#!pip install torch torchtext sentencepiece datasets wandb

In [103]:
import numpy as np
import torch
from torch import nn
import sys
import os
import math
import einops

sys.path.append(os.path.abspath("../../data"))
sys.path.append(os.path.abspath("../../nnets"))
from net_utils import get_module_list

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
SP_VOCAB_SIZE = 5000
TRAIN_SIZE = 5000

In [5]:
from text_data import CNNDatasetDecoderOnly

class Wrapper(CNNDatasetDecoderOnly):
    split_lengths = [TRAIN_SIZE, math.floor(TRAIN_SIZE * .1), 100]
    x_length = 15
    target_length = 15

wrapper = Wrapper(SP_VOCAB_SIZE)
datasets = wrapper.generate_datasets(BATCH_SIZE)
train = datasets["train"]
valid = datasets["validation"]

Found cached dataset cnn_dailymail (/Users/vik/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
100%|██████████| 3/3 [00:00<00:00, 42.35it/s]
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tokens.txt --model_prefix=cnn_dailymail --vocab_size=5000 --model_type=unigram
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: tokens.txt
  input_format: 
  model_prefix: cnn_dailymail
  model_type: UNIGRAM
  vocab_size: 5000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piec

In [110]:
# Add in ROPE embedding
class ROPE(nn.Module):
    def __init__(self, embedding_dim, seq_len):
        super(ROPE, self).__init__()
        self.embedding_dim = embedding_dim
        self.seq_len = seq_len

        self.cos_embeds = torch.zeros(seq_len, embedding_dim, device=DEVICE)
        self.sin_embeds = torch.zeros(seq_len, embedding_dim, device=DEVICE)

        embed_pos = 10000 ** (-2 * torch.ceil((torch.arange(0, embedding_dim) + 1) / 2) / embedding_dim)
        for i in range(0, seq_len):
            self.cos_embeds[i] = torch.cos((i + 1) * embed_pos)
            self.sin_embeds[i] = torch.sin((i + 1) * embed_pos)

        self.indices = torch.zeros(self.embedding_dim, device=DEVICE, dtype=torch.long)
        self.mask = torch.zeros(self.embedding_dim, device=DEVICE, dtype=torch.int)
        for i in range(0, embedding_dim, 2):
            self.indices[i] = i + 1
            self.indices[i+1] = i

            self.mask[i] = -1
            self.mask[i+1] = 1


    def rotate(self, x):
        return x[...,self.indices] * self.mask

    def forward(self, x):
        current_val = x * self.cos_embeds[:x.shape[-2],:]
        next_val = self.rotate(x) * self.sin_embeds[:x.shape[-2],:]
        return current_val + next_val


class MultiHeadAttention(nn.Module):
    def __init__(self, input_units, attention_heads, mask=False):
        super(MultiHeadAttention, self).__init__()
        self.input_units = input_units
        self.attention_heads = attention_heads
        self.head_units = int(input_units/attention_heads)
        self.mask = mask

        k = math.sqrt(1/self.input_units)
        # Drop bias
        # Single kv head
        self.kv_proj_weight = nn.Parameter(torch.rand(2, input_units, self.head_units) * 2 * k - k)
        self.q_proj_weight = nn.Parameter(torch.rand(input_units, self.attention_heads * self.head_units) * 2 * k - k)
        self.out_proj_weight = nn.Parameter(torch.rand(self.attention_heads * self.head_units, input_units) * 2 * k - k)

        # 1024 is max sequence length
        self.rope = ROPE(self.head_units, 1024)

    def forward(self, queries, keys, values):
        # convert to 4d tensor with batch_size, attn_heads, seq_len, embedding_dim
        proj_queries = torch.einsum("...se, eo->...so", queries, self.q_proj_weight)
        proj_queries = proj_queries.view(queries.shape[0], queries.shape[1], self.attention_heads, self.head_units).swapaxes(1,2)
        proj_queries = self.rope(proj_queries)

        proj_keys = torch.einsum("...se, eo->...so", keys, self.kv_proj_weight[0])
        proj_keys = proj_keys.view(keys.shape[0], keys.shape[1], self.head_units)
        proj_keys = self.rope(proj_keys)

        proj_values = torch.einsum("...se, eo->...so", values, self.kv_proj_weight[0])
        proj_values = proj_values.view(values.shape[0], values.shape[1], self.head_units)

        attention = torch.einsum("baqh, bhk->baqk", proj_queries, torch.transpose(proj_keys, -1, -2)) / np.sqrt(proj_keys.shape[-1])
        if self.mask:
            # Prevent decoder queries from looking at tokens that come after
            # Do this by setting attention to negative infinity, so it is softmaxed to zero in the next step
            mask = torch.full((attention.shape[-2], attention.shape[-1]), -torch.inf, device=DEVICE)
            attention += torch.triu(mask, diagonal=1)

        # Softmax on last dimension
        # Sequence-wise softmax, so attention between one sequence and other sequences sums to 1
        attention = torch.softmax(attention, dim=-1)
        weighted_values = torch.einsum("baqk, bke->baqe", attention, proj_values)

        # Swap attention head and sequence axis, then reshape to batch, seq, embedding
        weighted_values = weighted_values.swapaxes(1,2).reshape(queries.shape[0], queries.shape[1], -1)
        weighted_values = torch.einsum("...se, eo->...so", weighted_values, self.out_proj_weight)
        return weighted_values

In [111]:
class SwiGLU(nn.Module):
    def __init__(self, input_units, hidden_units):
        super(SwiGLU, self).__init__()
        self.linear1 = nn.Linear(input_units, hidden_units, bias=False)
        self.linear2 = nn.Linear(input_units, hidden_units, bias=False)
        self.linear3 = nn.Linear(hidden_units, input_units, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x1 = self.linear1(x)
        swish = self.sigmoid(x1) * x1
        x2 = self.linear2(x)
        swiglu = self.linear3(swish * x2)
        return swiglu

class DecoderBlock(nn.Module):
    def __init__(self, input_units, attention_heads, hidden_units=2048, dropout_p=.1):
        super(DecoderBlock, self).__init__()
        self.in_attn = MultiHeadAttention(input_units, attention_heads, mask=True)
        self.dropouts = get_module_list(2, nn.Dropout, dropout_p)
        # Drop bias
        self.lns = get_module_list(2, nn.LayerNorm, input_units)
        # Switch to swiglu from two linear layers
        self.swiglu = SwiGLU(input_units, hidden_units)

    def forward(self, x):
        weighted_values = self.dropouts[0](self.in_attn(x, x, x))
        # Pre normalization
        x = x + self.lns[0](weighted_values)

        reprojected = self.dropouts[1](self.swiglu(x))
        # Pre normalization
        x = x + self.lns[1](reprojected)
        return x

In [112]:
class Transformer(nn.Module):
    def __init__(self, input_units, hidden_units, attention_heads, padding_idx, max_len=256, blocks=1):
        super(Transformer, self).__init__()
        self.blocks = blocks
        self.output_embedding = nn.Linear(hidden_units, input_units)
        self.embedding = nn.Embedding(input_units, hidden_units, padding_idx=padding_idx)
        self.dropouts = get_module_list(2, nn.Dropout, .1)
        self.decoders = get_module_list(blocks, DecoderBlock, hidden_units, attention_heads)

    def forward(self, x):
        dec_outputs = self.dropouts[1](self.embedding(x))
        for i in range(self.blocks):
            dec_outputs = self.decoders[i](dec_outputs)

        token_vectors = self.output_embedding(dec_outputs)
        return token_vectors

In [113]:
def generate(sequence, pred, target, wrapper):
    prompts = wrapper.decode_batch(sequence[:,:wrapper.x_length].cpu())
    texts = wrapper.decode_batch(torch.argmax(pred[:,wrapper.x_length:], dim=2).cpu())
    correct_texts = wrapper.decode_batch(target[:,wrapper.x_length:].cpu())

    displays = []
    for p, t, ct in zip(prompts, texts, correct_texts):
        displays.append(f"{p} | {ct} | {t}")
    return displays

In [114]:
from tqdm.auto import tqdm
import wandb
import time

wandb.init(project="decoder-only", notes="ROPE embedding", name="rope-embed")

# TODO: Profile and improve perf - https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html
model = Transformer(wrapper.vocab_size, 512, 8, blocks=1, padding_idx=wrapper.pad_token).to(DEVICE)
loss_fn = nn.CrossEntropyLoss(ignore_index=wrapper.pad_token)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
wandb.watch(model, log_freq=100)

[]

In [115]:
EPOCHS = 100
DISPLAY_BATCHES = 2
OUT_SEQUENCE_LEN = wrapper.y_length
PRINT_VALID = True
ACCUMULATE_STEPS = 1

for epoch in range(EPOCHS):
    # Run over the training examples
    train_loss = 0
    match_pct = 0
    optimizer.zero_grad(set_to_none=True)
    start = time.time()
    for batch, (sequence, target, prev_target) in tqdm(enumerate(train)):
        pred = model(sequence.to(DEVICE))

        # If you use a batch, need to reshape pred to be batch * sequence, embedding_len to be compatible
        # Similar reshape with target to be batch * sequence vector of class indices
        cpred = pred[:, wrapper.x_length:]
        ctarget = target[:, wrapper.x_length:]
        loss = loss_fn(cpred.reshape(-1, cpred.shape[-1]), ctarget.reshape(-1).to(DEVICE))
        loss.backward()
        train_loss += loss.item()

        # Accumulate gradients
        # This seems to perform worse than no accumulation over a
        # small data set.  Test with larger set.
        if batch % ACCUMULATE_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
    end = time.time()

    with torch.no_grad():
        mean_loss = train_loss / len(train) / BATCH_SIZE
        wandb.log({"loss": mean_loss, "epoch_time": end - start})
        print(f"Epoch {epoch} train loss: {mean_loss}")
        sents = generate(sequence, pred, target, wrapper)
        for sent in sents[:DISPLAY_BATCHES]:
            print(sent)

        if PRINT_VALID and epoch % 10 ==0:
            # Compute validation loss.  Unless you have a lot of training data, the validation loss won't decrease.
            valid_loss = 0
            # Deactivate dropout layers
            model.eval()
            for batch, (sequence, target, prev_target) in tqdm(enumerate(valid)):
                # Inference token by tokens
                outputs = sequence[:,:(wrapper.x_length + 1)].to(DEVICE)
                # TODO: Investigate memory leak with valid generation
                for i in range(OUT_SEQUENCE_LEN):
                    pred = model(outputs)
                    last_output = torch.argmax(pred, dim=2)
                    outputs = torch.cat((outputs, last_output[:,-1:]), dim=1)

                cpred = pred[:, wrapper.x_length:]
                ctarget = target[:, wrapper.x_length:]
                loss = loss_fn(cpred.reshape(-1, cpred.shape[-1]), ctarget.reshape(-1).to(DEVICE))
                valid_loss += loss.item()
            mean_loss = valid_loss / len(valid) / BATCH_SIZE
            wandb.log({"valid_loss": mean_loss})
            print(f"Valid loss: {mean_loss}")
            sents = generate(sequence, pred, target, wrapper)
            for sent in sents[:DISPLAY_BATCHES]:
                print(sent)
            # Reactivate dropout
            model.train()

145it [00:11, 12.65it/s]


Epoch 0 train loss: 0.19722925064892605
Demonstrators storm a hotel Saturday where Asian leaders | were to meet . Thousands of "red shirt" protesters | . .ld . Ones say theD
Report: 38 percent increase in cancer in developing countries by 2030 | . Emerging economies such as China, Russia and India will | . Heirates


2it [00:00,  4.67it/s]


Valid loss: 0.23285993188619614
Lindsey Vonn notches the 65th World Cup win | of her skiing career . American has an eight-point lead | . The threes say s tolds
The Rock and Roll Hall of Fame announces the presenters for | its 2015 induction . Paul McCartney will introduce Ring | . The threes to


145it [00:11, 12.56it/s]


...

Epoch 8 train loss: 0.01989970178311241
Stephan Zielinski composed a song based on a s | wine flu gene . He created an algorithm that as | wine flu gene . Mo created an algorithm, as
The Marines' Toys for Tots program has distributed | donated toys for 60 years . Bad economy has resulted | donate . ys for 60 years . FBId has has resulted music


145it [00:11, 12.73it/s]


Epoch 9 train loss: 0.015781363459496663
Man convicted of murder of teenage actor who starred in new " | Harry Potter" film . Karl Bishop, 22, attack | Harry Pottere film . Karl Bishop, 22, attack on
Czech Republic assumed six-month presidency of European Union this | month . Czech artists has apologized for artwork that has | month . Czech artists has apologized for artwork that has


50it [00:04, 12.35it/s]


KeyboardInterrupt: 

In [ ]:

from torchinfo import summary

print(summary(model))

In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity

with profile(activities=[ProfilerActivity.CPU], record_shapes=True, ) as prof:
    model(sequence.to(DEVICE), prev_target.to(DEVICE))

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

In [102]:
from einops import rearrange

x = torch.arange(0,10)


tensor([-1,  0, -3,  2, -5,  4, -7,  6, -9,  8])